# F.L.A.T. (FlawLess AgenTs)

Inspired on Anthropic's scholarly tome about [**building effective agents**.](https://www.anthropic.com/research/building-effective-agents)



Let's get started, here we will show you how to use LLM's to build AI Apps and Agents using normal python logic and workflows. For that we will load the library and a dummy email object we will use in the tutorial.


In [1]:

from datetime import date
from flat_ai import FlatAI, configure_logging
from typing import List
from pydantic import BaseModel
from dotenv import load_dotenv
import os

load_dotenv()

userdata = {
    'openai_key': os.getenv('OPENAI_API_KEY')
}
# we will be using a llama3.1 on together model here, but you can use openai, ollama, etc
#llm = FlatAI(api_key=userdata.get('together_api_key'), base_url = 'https://api.together.xyz/v1',  model='meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo')
# openai example:
llm = FlatAI(api_key=userdata.get('openai_key'),   model='gpt-4o-mini')


## Logic Blocks

The idea is that you can, program with LLMs, Like you would in any Python program:

### 1. LLM Gates
You want to use an LLM to decide whats the next step:
Let's see if our email is urgent

In [14]:

email_body = 'Hello, would love to schedule a time to talk about the urgent project deadline next week. Can we meet tomorrow? also can you message joe about our meeting'

if llm.is_true('is this email urgent?', email=email_body):
    print("🚨 Drop everything! We've got an urgent situation here!")
else:
    print("😌 Relax, it can wait until after coffee")

🚨 Drop everything! We've got an urgent situation here!


## 2. Workflow: Routing: Switch Case

Similar to if/else statements, but for when your LLM needs to be more dramatic with its life choices.

*For example*, let's say we want to classify a message into different categories:

In [15]:
options = {
    'meeting': 'book a meeting',
    'spam': 'people trying to sell you stuff you dont want',
    'other': 'this sounds like something else'
}

match llm.classify(options, email=email_body):
    case 'meeting':
        print("📅 Time to book a meeting!")
        llm.add_context(meeting=True)
    case 'spam':
        print("🚫 No, I don't want to extend my car's warranty")
    case 'other':
        print("🤔 Interesting... but what does it mean?")

📅 Time to book a meeting!


### 3. Objects
Need your LLM to fill out objects like a trained monkey with a PhD in data entry? Just define the shape and watch the magic! 🐒📝


In [16]:
# let's turn our email_body into an object first
class Email(BaseModel):
    to_email: str
    from_email: str
    body: str
    subject: str

email = llm.get_object(Email, email_body=email_body, instruction='make up the attributes you cannot find in the body')
print(email)

to_email='jane.doe@example.com' from_email='john.smith@example.com' body='Hello, would love to schedule a time to talk about the urgent project deadline next week. Can we meet tomorrow? Also, can you message Joe about our meeting.' subject='Meeting Request to Discuss Project Deadline'



Want a list of things? Just throw a schema at it and watch it spin like a hamster on a crack coated wheel.

For example: Time to extract those action items like we're mining for AI gold!



In [17]:

class ActionItem(BaseModel):
    action: str
    description: str
    status: str
    priority: str
    due_date: str
    assignee_name: str
    assignee_email: str

# we can set the context globally so we dont have to pass it every time
llm.set_context(email=email, today = date.today() )
if llm.is_true('are there action items in this email?'):
    print("🎯 Found some action items:")
    for action_item in llm.get_object(List[ActionItem]):
        print(f"\n🔸 Action: {action_item.action}")
        print(f"\n🔸 Action Description: {action_item.description}")
        print(f"  Priority: {action_item.priority}")
        print(f"  Due: {action_item.due_date}")
llm.clear_context()

🎯 Found some action items:

🔸 Action: Schedule a meeting

🔸 Action Description: Discuss the urgent project deadline next week.
  Priority: High
  Due: 2025-02-12

🔸 Action: Message Joe about the meeting

🔸 Action Description: Inform Joe regarding the scheduled meeting about the project deadline.
  Priority: Medium
  Due: 2025-02-12


### 4. Function calling and Parallelism

Let the LLM choose between sending an email and or a calendar invite - what could possibly go wrong? 🎲

In [18]:
def send_email(name: str, email_address_list: List[str], subject: str, body: str):
    print(f"""\n=====\nTask:📧 Sending email to {name}
            To: {', '.join(email_address_list)}
            Subject: {subject}
            Body: {body}""")
    return 'email sent'

def send_calendar_invite(subject: str, meeting_date: str, location: str, attendees: List[str]):
    print(f"""\n=====\nTask:Sending calendar invites")
            Subject: {subject}
            Date: {meeting_date}""")
    return 'invite sent'

instructions = """extract list of action items and call the funcitons required"""

functions_to_call = llm.get_functions([send_calendar_invite, send_email], instructions = instructions, email=email, current_date = date.today())
for func in functions_to_call:
    print(f"-function to call: {func.function}, and args:{func.arguments}")
# lets call them all in tandem
functions_to_call() # call all the functions in parallel

-function to call: <function send_calendar_invite at 0x1079e1800>, and args:{'subject': 'Meeting Request to Discuss Project Deadline', 'meeting_date': '2025-02-12', 'location': 'Virtual Meeting', 'attendees': ['jane.doe@example.com', 'john.smith@example.com']}
-function to call: <function send_email at 0x1079e0e00>, and args:{'name': 'Jane Doe', 'email_address_list': ['joe@example.com'], 'subject': 'Meeting Reminder', 'body': 'Hi Joe, just a reminder about the meeting scheduled for tomorrow with John regarding the urgent project deadline.'}

=====
Task:Sending calendar invites")
            Subject: Meeting Request to Discuss Project Deadline
            Date: 2025-02-12

=====
Task:📧 Sending email to Jane Doe
            To: joe@example.com
            Subject: Meeting Reminder
            Body: Hi Joe, just a reminder about the meeting scheduled for tomorrow with John regarding the urgent project deadline.


['invite sent', 'email sent']

### 7. Simple String Response
Sometimes you just want a straight answer - how refreshing! 🎯

In [19]:
subject = llm.get_string('what is the subject of the email?')
print(f"Email subject: {subject}")

Email subject: To provide an appropriate response, I would need more context about the content or the purpose of the email you are referring to. Could you please provide more details?


### 8. Streaming Response
Watch the AI think in real-time - it's like watching paint dry, but with more hallucinations! 🎬

In [6]:
print("Generating response...")
for chunk in llm.get_stream('write a polite response to this email'):
    print(chunk, end='')
print(f"\nfull_resonse:{chunk.string}")

Generating response...
Of course! Please provide the email you’d like me to respond to, and I'll help you craft a polite response.
full_resonse:Of course! Please provide the email you’d like me to respond to, and I'll help you craft a polite response.


## 9. Code Response 
Some other times, you want to ask your LLM to generate code

In [7]:
code_object = llm.get_code("a function to count and print the number of rs in the word strawberry", as_stream = False, requirements="just a function, no example_code")
print(code_object)
print(f"\nGENERATED CODE:\n {code_object.raw_code}")
print(f"\nCODE NOTES: {code_object.code_notes}\n")

#run the code using exec or eval
exec(code_object.raw_code)

PythonCodeObject <obj.code_notes>, <obj.raw_code>

GENERATED CODE:
 def count_r_in_strawberry():
    word = "strawberry"
    count_r = word.count('r')
    print("Number of 'r' in 'strawberry':", count_r)

CODE NOTES: This function counts the occurrences of the letter 'r' in the word 'strawberry' and prints the result.



#### Code as a stream  
want to get it as a stream?

In [2]:
for chunk in llm.get_code_as_stream("a function to print the number of rs in the word strawberry and code to call the function", requirements="just a function, no example_code"):  
    print(chunk, end='')
print("\nNow we can inspect the object")
code_object = chunk # last chunk is a code object
print(f"\nGENERATED CODE:\n {code_object.raw_code}")
print(f"\nCODE NOTES: {code_object.code_notes}\n")

#run the code using exec or eval
exec(code_object.raw_code)

```python
def count_r_in_strawberry():
    word = "strawberry"
    count = word.count('r')
    print(count)

# Calling the function
count_r_in_strawberry()
```PythonCodeObject <obj.code_notes>, <obj.raw_code>
Now we can inspect the object

GENERATED CODE:
 def count_r_in_strawberry():
    word = "strawberry"
    count = word.count('r')
    print(count)

# Calling the function
count_r_in_strawberry()

CODE NOTES: This function counts and prints the number of occurrences of the letter 'r' in the word 'strawberry'.

3


## 🎉 Tada!

And there you have it, ladies and gents! You're now equipped with the power to boss around LLMs like a project manager remotely working from Ibiza.

Now off you go, forth and build something that makes ChatGPT look like a calculator from 1974! Just remember - if your AI starts humming "Daisy Bell" while slowly disconnecting your internet... well, you're on your own there, buddy! 😅